## Description
With this notebook, NNI experiments can be run taking advantage of hosted runtimes by Google. Particularly, they will exploit GPU to speed up the execution.<br>
<i>To enable GPU usage: navigate to Edit→Notebook Settings then select GPU from the Hardware Accelerator drop-down.</i><br>
Due to time constraints imposed to such free resources, we have to accept some trade-off and to design some workaround in order to get as much as possible benefit from the Colab tool.<br>
Additionally, with the aim of fully monitoring the experiments through the graphical interface provided by NNI, we will use ngrok to overcome the fact that Colab doesn’t expose its public IP and ports thus hindering access to the NNI web UI.

<b>Step 0</b><br>
Have a look at what NNI is and how to use it in Google Colab:<br>
&emsp; https://nni.readthedocs.io/en/v2.2/Overview.html<br>
&emsp; https://nni.readthedocs.io/en/stable/sharings/nni_colab_support.html<br>

## Download the course-dedicated folder and install/import packages

In [1]:
%%capture
! pip uninstall --yes gdown
! pip install gdown==4.3.1
#! pip install gdown -U --no-cache-dir
! pip uninstall --yes numpy
! pip install numpy==1.19.5
! pip uninstall --yes tensorflow

import os
import gdown
from gdown.download_folder import download_and_parse_google_drive_link
import time
import datetime
from google.colab import files
from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/MLinApp_project_mine/NNI_ste
!ls

/content/drive/.shortcut-targets-by-id/1wXg2dci4kAWzFZAvhlShcmwk3t0dUHTP/MLinApp_project_mine/NNI_ste
configurations		      post-optimization_MLinApp.ipynb  results
experiments		      postopt.py		       root
ngrok			      __pycache__		       searchspaces
ngrok-stable-linux-amd64.zip  requirements_pip_old.txt
output			      requirements_pip.txt


In [4]:
%%capture
import os
if "requirements_pip.txt" not in os.listdir("./"):
  raise SystemError('requirements_pip.txt missing!')
else:
  ! pip install -r requirements_pip.txt

## Download ngrok and save your personal token

In [5]:
!rm -r ngrok
!rm -r ngrok-stable-linux-amd64.zip

Once you get it after signing up, copy in the cell below your ngrok personal token.

In [6]:
ngrok_personal_token = "29iBKME1LpDpmkf2eys4fnL5lxM_31pDquiHicHmc2EG6rpgK"

In [7]:
%%capture
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [8]:
! unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [9]:
! ./ngrok authtoken $ngrok_personal_token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## Check the correct TensorFlow version (2.5.0) is installed and GPU availability

In [10]:
import tensorflow as tf

print("TF version: {}\n".format(tf.__version__))

#tf.config.list_physical_devices('GPU')
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found. This might be a CPU-based runtime. Be sure GPU is not required for the upcoming NNI experiment.')
else:
  print("GPU check succeeded!")
  print('GPU index: {}'.format(device_name[device_name.rfind(":")+1:]))

TF version: 2.5.0

GPU check succeeded!
GPU index: 0


## Start NNI experiment

**NOTE that the dataset information have to be included within the .py file**

In [11]:
from google.colab.output import eval_js

js = """
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#toolbar-add-code").click()
}setInterval(ClickConnect,300000)
"""

eval_js(js)

1

Select which network architecture will be considered in the NNI experiment

In [12]:
net_type = "scnn" #Usare CNN inizialmente e poi Cambiare in scnn

configuration_filename = "nni_"+net_type+"_trial.yml"

if net_type == "scnn":
  print("REMEMBER that a previous CNN experiment is needed to have a reference network structure.")

REMEMBER that a previous CNN experiment is needed to have a reference network structure.


Start the NNI experiment

In [14]:
! nnictl create --config configurations/$configuration_filename --port 5000 &
get_ipython().system_raw('./ngrok http 5000 &')

INFO:  expand searchSpacePath: ../searchspaces/nni_SearchSpace_scnn.json to /content/drive/.shortcut-targets-by-id/1wXg2dci4kAWzFZAvhlShcmwk3t0dUHTP/MLinApp_project_mine/NNI_ste/configurations/../searchspaces/nni_SearchSpace_scnn.json 
INFO:  expand codeDir: ../experiments/ to /content/drive/.shortcut-targets-by-id/1wXg2dci4kAWzFZAvhlShcmwk3t0dUHTP/MLinApp_project_mine/NNI_ste/configurations/../experiments/ 
ERROR: Port 5000 is used by another process, please reset the port!
You could use 'nnictl create --help' to get help information


## View the NNI experiment
Open the first displayed link to access the NNI web UI

In [15]:
! curl -s http://localhost:4040/api/tunnels # don't change the port number 4040

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://82c3-35-187-239-13.ngrok.io","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://82c3-35-187-239-13.ngrok.io","proto":"http","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


## Stop running NNI experiment

In [ ]:
! nnictl stop

INFO:  Stopping experiment iFxzTA94
INFO:  Stop experiment success.


In [ ]:
folder = "NNI_ste"

experiment_IDs = [ii for ii in os.listdir(os.path.expanduser('~')+"/nni-experiments") if ii[0]!="."]

experiment_folders = [os.path.join(os.path.expanduser('~'),"nni-experiments", jj) for jj in experiment_IDs]

experiment_folders.sort(key=lambda x: os.path.getmtime(x))

exp_ID = experiment_folders[-1][experiment_folders[-1].rfind("/")+1:]

print("Current experiment ID: {}".format(exp_ID))

local_experiment_folder = f"/content/drive/MyDrive/MLinApp_project_mine/{folder}/results/Experiment_{net_type}_{exp_ID}/"


print("Local experiment folder: {}".format(local_experiment_folder))

! mkdir $local_experiment_folder

Current experiment ID: QHj6Ximx
Local experiment folder: /content/drive/MyDrive/MLinApp_project_mine/NNI_ste/results/Experiment_scnn_QHj6Ximx/


## Periodically download experiment data

In [ ]:
def zip_folder(folder_path):
  final = ""
  for dirname in os.listdir(folder_path):
      if dirname.startswith(f'tmp_{net_type}_{exp_ID}'):
        final = dirname
  now = datetime.datetime.now().strftime("%Y-%m-%d_%H.%M.%S") # UTC+0 time
  destination = folder_path+"_"+now+".zip"
  origin = folder_path+f"/{final}"
  print(origin)
  ! zip -r $destination $origin &> /dev/null
  return destination
def download_output():
  path = "output"
  source = zip_folder(path)
  while True:
    if source[source.rfind("/")+1:] in os.listdir("./"):
      break
  destination = local_experiment_folder
  #destination = f"/content/drive/MyDrive/NNI_ste2/results/Experiment_"+net_type+"_"+exp_ID+"/"
  ! scp $source $destination

  os.remove(source)
  for ii in os.listdir(destination):
    if ("output" in ii) & (ii != source[source.rfind("/")+1:]):
      os.remove(destination+ii)
def download_experiment():
  path = os.path.expanduser('~')+"/nni-experiments/"+exp_ID
  source = zip_folder(path)
  while True:
    if source[source.rfind("/")+1:] in os.listdir(os.path.expanduser('~')+"/nni-experiments/"):
      break
  destination = local_experiment_folder
  # destination = f"/content/drive/MyDrive/NNI_ste2/results/Experiment_"+net_type+"_"+exp_ID+"/"

  ! scp $source $destination
  os.remove(source)
  for ii in os.listdir(destination):
    if (exp_ID in ii) & (ii != source[source.rfind("/")+1:]):
      os.remove(destination+ii)

In [ ]:
timer_download = 5*60 # minutes*seconds

t0 = time.time()
ready2download = False
while True:
  t1 = time.time()
  if t1-t0 >= timer_download:
    t0 = t1
    ready2download = True
    #break
  if ready2download:
    download_output()
    download_experiment()
    ready2download = False
    print(time.asctime())

output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:16:45 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:21:45 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:26:46 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:31:45 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:36:46 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:41:46 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:46:46 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:51:46 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 12:56:46 2022
output/tmp_scnn_QHj6Ximx_miRNA-superclass4
/root/nni-experiments/QHj6Ximx/
Sun Dec 11 13:01:47 2022


KeyboardInterrupt: ignored

In [ ]:
download_output()
download_experiment()



output/tmp_scnn_BYb0Vo26_miRNA-superclass3
/root/nni-experiments/BYb0Vo26/


## List running experiments

In [ ]:
! nnictl experiment list --all

----------------------------------------------------------------------------------------
                Experiment information
Id: 3hd0A51T    Name: MLinApp    Status: STOPPED    Port: 5000    Platform: local    StartTime: 2022-11-08 20:23:47    EndTime: 2022-11-08 20:38:20
Id: m4A7xwMH    Name: MLinApp    Status: STOPPED    Port: 5000    Platform: local    StartTime: 2022-11-08 20:38:29    EndTime: 2022-11-08 20:39:06
Id: PtNn130S    Name: MLinApp    Status: STOPPED    Port: 5000    Platform: local    StartTime: 2022-11-08 20:39:37    EndTime: 2022-11-08 20:43:05

----------------------------------------------------------------------------------------

